# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import tree
from google.colab import files
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

# Carga de los data sets de test y de entrenamiento

In [2]:
test = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Train_TP2_Datos_2020-2C.csv')

train_set_features = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/train_set_Features.csv')
test_set_features = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/test_set_Features.csv')

In [3]:
train_set_features = train_set_features.drop_duplicates(subset="Opportunity_ID")
test_set_features = test_set_features.drop_duplicates(subset="Opportunity_ID")

# Separamos la variable a predecir y los features

In [ ]:
#X = DATA , Y = TARGET 
X1, y1 = train_set_features[['Total_Taxable_Amount_USD',
                            'Bureaucratic_Code_0_Approved', 
                            'Products_Per_Op',
                            'TRF_Per_Op',
                            'Daily_Dif_Account_Created_Opportunity_Created_Date',
                            'Daily_Dif_Delivery_Start_Delivery_End_Date',
                            'Opp_Owner_Did_Last_Modifiy',
                            'Pricing, Delivery_Terms_Approved',
                            'Bureaucratic_Code_0_Approval',
                            'Last_Modified_date>Planned_Delivery_End_Date',
                            'Op_Has_A_Valid_Quote_Expiry_Date']], train_set_features['Stage']

Dividimos el set de entrenamiento en train y test, utilizando una proporción del 30% del total de los datos para el segundo.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=45)

# XGBoost

In [ ]:
xg_clas = XGBClassifier(colsample_bytree = 0.5, 
                        learning_rate = 0.27,
                        max_depth = 5, 
                        alpha = 10, 
                        n_estimators = 40)

In [ ]:
xg_clas.fit(X_train,y_train)


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.27, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=40, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
predicciones_train = xg_clas.predict_proba(X_test)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, predicciones_train)

0.45245900561092744

In [ ]:
xg_clas.feature_importances_

array([0.1385736 , 0.02691933, 0.06681917, 0.23792483, 0.06840276,
       0.07284367, 0.13044953, 0.05689373, 0.05352357, 0.08472418,
       0.06292567], dtype=float32)

In [ ]:
predicciones_finales = xg_clas.predict_proba(test_set_features[['Total_Taxable_Amount_USD',
                            'Bureaucratic_Code_0_Approved', 
                            'Products_Per_Op',
                            'TRF_Per_Op',
                            'Daily_Dif_Account_Created_Opportunity_Created_Date',
                            'Daily_Dif_Delivery_Start_Delivery_End_Date',
                            'Opp_Owner_Did_Last_Modifiy',
                            'Pricing, Delivery_Terms_Approved',
                            'Bureaucratic_Code_0_Approval',
                            'Last_Modified_date>Planned_Delivery_End_Date',
                            'Op_Has_A_Valid_Quote_Expiry_Date']])

In [ ]:
predicciones_exito = []
for prob in predicciones_finales:
  predicciones_exito.append(prob[1])

In [ ]:
csv_kaggle = pd.DataFrame()
csv_kaggle['Opportunity_ID'] = test_set_features["Opportunity_ID"]
csv_kaggle['target'] = predicciones_exito

In [ ]:
csv = csv_kaggle.to_csv('predicciones_xgb_features_feature_engineering_03.csv', index=False)
files.download('predicciones_xgb_features_feature_engineering_03.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Resultados

Algoritmo: **XGBoost**

Búsqueda de hiper parámetros: Aleatorio

Features analizados: 

+ Total_Taxable_Amount_USD
+ Bureaucratic_Code_0_Approved
+ Products_Per_Op
+ TRF_Per_Op
+ Daily_Dif_Account_Created_Opportunity_Created_Date               
+ Daily_Dif_Delivery_Start_Delivery_End_Date
+ Opp_Owner_Did_Last_Modifiy
+ Pricing, Delivery_Terms_Approved
+ Bureaucratic_Code_0_Approval
+ Last_Modified_date>Planned_Delivery_End_Date
+ Op_Has_A_Valid_Quote_Expiry_Date'

Hiper parámetros: 

- colsample_bytree = 0.5
- learning_rate = 0.27
- max_depth = 5
- alpha = 10
- n_estimators = 40

**Resultado Kaggle = 0,57150**

## Random Search

In [ ]:
xg_clas = XGBClassifier(eval_metric = 'mlogloss')

In [ ]:
from sklearn.metrics import make_scorer

posibles_hiperparametros = {'learning_rate':np.arange(0.01,0.3,0.1),
                              'n_estimators':np.arange(5,100,5),
                              'scale_pos_weight':np.arange(2,6,1),
                              'max_depth':np.arange(2,7,1),
                            'min_child_weight':np.arange(1,10,1),
                              'gamma':np.arange(0,0.5,0.1),
                              'subsample':np.arange(0.2,1,0.1),
                            'colsample_bytree':np.arange(0.4,0.8,0.05),
                              'colsample_bylevel':np.arange(0.4,0.8,0.05)}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

random_hiperparametros = RandomizedSearchCV(xg_clas, 
                                            posibles_hiperparametros, 
                                            n_iter = 100,
                                            verbose = 3,
                                            n_jobs = -1,
                                            scoring=LogLoss)
mejores_hp = random_hiperparametros.fit(X_train, y_train)

NameError: ignored

In [ ]:
mejores_hp.best_params_

{'colsample_bylevel': 0.55,
 'colsample_bytree': 0.75,
 'gamma': 0.30000000000000004,
 'learning_rate': 0.21000000000000002,
 'max_depth': 5,
 'min_child_weight': 2,
 'n_estimators': 50,
 'scale_pos_weight': 3,
 'subsample': 0.9000000000000001}

In [ ]:
predicciones_mejores_hp_train = mejores_hp.predict_proba(X_test)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, predicciones_mejores_hp_train)

0.4472716532467369

In [ ]:
predicciones_mejores_hp = random_hiperparametros.predict_proba(test_set_features[['Total_Taxable_Amount_USD',
                            'Bureaucratic_Code_0_Approved', 
                            'Products_Per_Op',
                            'TRF_Per_Op',
                            'Daily_Dif_Account_Created_Opportunity_Created_Date',
                            'Daily_Dif_Delivery_Start_Delivery_End_Date',
                            'Opp_Owner_Did_Last_Modifiy',
                            'Pricing, Delivery_Terms_Approved',
                            'Bureaucratic_Code_0_Approval',
                            'Last_Modified_date>Planned_Delivery_End_Date',
                            'Op_Has_A_Valid_Quote_Expiry_Date']])

In [ ]:
predicciones_mejores_hp_exito = []
for prob in predicciones_mejores_hp:
  predicciones_mejores_hp_exito.append(prob[1])

In [ ]:
csv_kaggle_mejores_hp = pd.DataFrame()
csv_kaggle_mejores_hp['Opportunity_ID'] = test_set_features["Opportunity_ID"]
csv_kaggle_mejores_hp['target'] = predicciones_mejores_hp_exito

In [ ]:
csv = csv_kaggle_mejores_hp.to_csv('predicciones_xgb_features_feature_engineering_04.csv', index=False)
files.download('predicciones_xgb_features_feature_engineering_04.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Resultados

Algoritmo: **XGBoost**

Búsqueda de hiper parámetros: Aleatorio

Features analizados: 

+ Total_Taxable_Amount_USD
+ Bureaucratic_Code_0_Approved
+ Products_Per_Op
+ TRF_Per_Op
+ Daily_Dif_Account_Created_Opportunity_Created_Date               
+ Daily_Dif_Delivery_Start_Delivery_End_Date
+ Opp_Owner_Did_Last_Modifiy
+ Pricing, Delivery_Terms_Approved
+ Bureaucratic_Code_0_Approval
+ Last_Modified_date>Planned_Delivery_End_Date
+ Op_Has_A_Valid_Quote_Expiry_Date'

Hiper parámetros: 

- colsample_bylevel: 0.55,
- colsample_bytree: 0.75,
- gamma: 0.30000000000000004,
- learning_rate: 0.21000000000000002,
- max_depth: 5,
- min_child_weight: 2,
- n_estimators: 50,
- scale_pos_weight: 3,
- subsample: 0.9000000000000001

**Resultado Kaggle = 0,55028**